# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

In [3]:
import sqlite3

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [4]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [5]:
cur.execute('''
CREATE TABLE Recipe(
id INTEGER PRIMARY KEY,
name VARCHAR(32) NOT NULL, 
minutes INTEGER, 
submitted DATE NOT NULL, 
description VARCHAR(180),
n_ingredients INTEGER)
''')

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [6]:
cur.execute('''
CREATE TABLE Review(id INTEGER PRIMARY KEY, 
user_id INTEGER NOT NULL, 
recipe_id INTEGER NOT NULL, 
date DATE NOT NULL, rating INTEGER NOT NULL, 
review VARCHAR(300), 
FOREIGN KEY (recipe_id) REFERENCES Recipe(id))
''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [7]:
import pandas as pd
reviews_sample = pd.read_csv('./data/reviews_sample.csv', sep=',')
reviews_sample = reviews_sample.set_index('Unnamed: 0')
recipes_sample_with_tags_ingredients = pd.read_csv('./data/recipes_sample_with_tags_ingredients.csv', sep=',')

for index, row in reviews_sample.iterrows():
    cur.execute("""
        INSERT INTO Review(id, user_id, recipe_id, date, rating, review) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, (index, row[0], row[1], row[2], row[3], row[4]))
con.commit()


for index, row in recipes_sample_with_tags_ingredients.iterrows():
    cur.execute("""
        INSERT INTO Recipe(id, name, minutes, submitted, description, n_ingredients) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, (row[1], row[0], row[2], row[4], row[6], row[7]))
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [8]:
cur.execute('''
SELECT * 
FROM Recipe 
WHERE n_ingredients = 10
''')
cur.fetchmany(5)

[(176, 'cola cake', 55, '1999-08-22', None, 10),
 (246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150,
  '1999-10-04',
  'brazilian bean soup',
  10),
 (373,
  'tiramisu   balducci s in new york city',
  95,
  '1999-08-17',
  'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.',
  10),
 (544, 'greek mushroom salad', 38, '1999-08-10', None, 10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [9]:
cur.execute('''
SELECT name, 
MAX(minutes) 
FROM Recipe
''')
cur.fetchone()

('strawberry liqueur', 129615)

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [13]:
recipe_id = int(input('Input recipe ID: '))
cur.execute('''
SELECT * FROM Recipe 
WHERE id = ?''', (recipe_id, ))
j = cur.fetchone()
if j is not None:
    print(j)
else:
    print('Not in data')

(176, 'cola cake', 55, '1999-08-22', None, 10)


8. Найдите кол-во отзывов с рейтингом 5.

In [15]:
cur.execute('''
SELECT COUNT(*) 
FROM Review 
WHERE rating = 5
''')
cur.fetchone()

(91361,)

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [16]:
cur.execute('''
    SELECT COUNT(*) OVER()
    FROM recipe
    LEFT JOIN review
    ON recipe.id = review.recipe_id
    GROUP BY recipe.id
    HAVING 0 = COUNT(review.rating < 4)
    LIMIT 1
''')
cur.fetchone()

(1900,)

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [17]:
cur.execute('''
    SELECT COUNT(*)
    FROM Recipe
    WHERE minutes >= 15
    AND submitted BETWEEN '2010-01-01' AND '2010-12-31'
''')
cur.fetchone()

(1319,)

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [18]:
cur.execute('''
    SELECT recipe.id, recipe.name, review.user_id, review.date, review.rating
    FROM recipe
    LEFT JOIN review
    ON recipe.id = review.recipe_id
    WHERE recipe.n_ingredients >= 3
    ORDER BY recipe.id
''')

cur.fetchall()

[(48, 'boston cream pie', 32421, '2002-03-15', 0),
 (48, 'boston cream pie', 68674, '2004-05-03', 2),
 (55, 'betty crocker s southwestern guacamole dip', 53959, '2006-01-12', 4),
 (55, 'betty crocker s southwestern guacamole dip', 165567, '2006-03-31', 5),
 (55, 'betty crocker s southwestern guacamole dip', 851190, '2010-05-23', 5),
 (55, 'betty crocker s southwestern guacamole dip', 1060485, '2009-04-07', 5),
 (66, 'black coffee barbecue sauce', 8679, '2003-03-17', 5),
 (66, 'black coffee barbecue sauce', 42938, '2002-10-21', 4),
 (66, 'black coffee barbecue sauce', 124416, '2010-07-08', 5),
 (66, 'black coffee barbecue sauce', 133174, '2010-07-04', 5),
 (66, 'black coffee barbecue sauce', 136004, '2008-06-05', 5),
 (66, 'black coffee barbecue sauce', 136813, '2007-04-14', 5),
 (66, 'black coffee barbecue sauce', 176798, '2010-10-02', 5),
 (66, 'black coffee barbecue sauce', 314792, '2008-02-04', 5),
 (66, 'black coffee barbecue sauce', 330545, '2009-01-02', 5),
 (66, 'black coffee ba